In [23]:

val charList = List("a", "b" , "c")
val lst2 = List(1,2,3)
val lst3 = lst1.zip(lst2)

val thisy = Map("d" -> 5)

val newMap = lst3.foldLeft[Map[String,Int]] (Map.empty) ( (acc, elt) => acc + elt )


// this is how you extend maps
val ext = thisy ++ newMap
println(ext)


Map(d -> 5, a -> 1, b -> 2, c -> 3)


charList: List[String] = List("a", "b", "c")
lst2: List[Int] = List(1, 2, 3)
lst3: List[(String, Int)] = List(("a", 1), ("b", 2), ("c", 3))
thisy: Map[String, Int] = Map("d" -> 5)
newMap: Map[String, Int] = Map("a" -> 1, "b" -> 2, "c" -> 3)
ext: Map[String, Int] = Map("d" -> 5, "a" -> 1, "b" -> 2, "c" -> 3)

# Youre going to redo your lettuce notes because you don't know what the fuck is going on lol

### Lettuce: Salad for CSCI 3155 

We will introduce our first language called lettuce, a language with let bindings. This langauge is inspired by an existing family of lanaguages called ML (meta langauages), which has been implemented in general purpose langauges such as SML, OCAML, ELM and so on. The Lettuce langauge will illustrate the following important concepts: 

    * Immutable data: environments
    * Function calls, closures and recursive functions. 
    * Mutable data through references. 
    
    
Lets build some example programs to understand the central concept in our let language: the 'let' binding. 


## Let Bindings

Let bindings allow us to bind an identifier to a value and use it inside of a body expression, using the syntax in the following form. 

let (symbolname) = (defining expression) in (body expression) 

Lets practice writing programs in Lettuce (no pun intended) 

### Example 1

    let x = 3.5 in x + x 
    
First of all, it defines a symbol named x. The defining expression here is 3.5, and thus evaluates to 3.5. The body expression is x + x. With x bound to 3.5, what does this evaluate too? The answer is 7.0 

Therefore the whole expression can be rewritten as follows: 

### Example 2 

We can now nest bindings as follows: 
```
    let x = 3.0 in 
        let y = 4.5 - x in 
            x + 2.0 * y      
```

This has two let bindings:

Outer binding: 
    let x = 3.0 in ((body expression #1)) 
    
defines the symbol x in the defining expression 3.0 and the body expression #1 is given by 

    let y = 4.5 in x + 2.0 * y 
    
defines the symbol 'y' to the defining expression 4.5 - x and the body expression #2 is given by x + 2.0 * y

Now that we have picked it apart, let us evaluate this expression ourselves. 

__eval__("let x=3.0 in let y=4.5 - x in x + 2.0y") -> 

__eval__("let y = 4.5 - 3.0 in 3.0 + 2.0y) -> 

__eval__(3.0 + 2.0*1.5) -> 

__eval__(6.0) --> 

6.0


### Example 3

Let us do one more differnetn nesting of let bindings. 
```

    let x = (let y = 5 in y * y ) in 
        let z = 15 * x in 
            x - z
```

Thus far we saw body expressions having let bindings. Now we can allow let bindings for the defining expressions as well. 

    let x = (defining expression #1) in (body expression #1) 
    
Defining expression #1 
    
        let y = 5 in y * y 
        
The body expression #1 is 
    
        let z = 15 * x in x - z 
        
How do we evaluate it? Now is a good time to break up the evaluation of this let binding         
    
        let sym = (defining expression) in (body expression)
        
as follows: 

    1. Evaluate the defining expression : eval(defining expression) 
    2. Associate the smybol sym with the result from step 1 
    3. Evaluate the body expression but now remember the association from step 1
    
    
### Let bindings in Scala

Scala does not have let bindings with the same syntax but it has let bindings through the 'val' declaration. 
```
    val x = 10 
    val y = x + 10 
    val z = y + 10 
    (x + y + z) 
```

Here we ahve assinged x to 10 and y to x + 10 and then z to y + 10 with x+y+z being the overall value of our code fragment above. 

Compare with how we could write something similar in Lettuce. 
```
    let x = 10 in 
        let.y = x + 10 in 
            let z = y + 10 in 
                x + y + z 
```
Here is another example in Scala: 
```
    val z = {
        val x = 10 
        val y = 15 
        x + y
    }
    
    return z + 10 
```

In Lettuce, we could would write 
```
    let z = {
        let x = 10 in 
            let y = 15 in 
                x + y 
                
    }
        in 
            z + 10 
```
m
Now you appreciate how let bindingsalso arise in scala without the let id = defining-expr in body-expr syntax 


## Scoping in Lettuce

It is now important to understand scoping rules. Scoping rules allows us to understand which let bindings applies to a given identifeier x. 

Consider the following progrma
```
    let y = 15 in 
        let x = 10 in x + y 
```
Consider the expression x + y. You can undesrtand which let binding has defined x an which let binding has defined y. 

However consider the more complicated case: 
```
    let x = 10 in 
    let x = 10 in 
    let x = x + 10 in 
        x + 10 
```

Here we are repeatedly defining x, however the right hand sides refer back to the left hand side. Is this allowed? The equivalent code in scala arises an error since a statement like val x = x + 10 
in sacla is forbidden. 

In [5]:
val x = 10 
val x = x + 10 
val x = x + 10 
x + 10

cmd5.sc:2: x is already defined as value x
val x = x + 10 
          ^cmd5.sc:3: x is already defined as value x
val x = x + 10 
          ^Compilation Failed

: 

However, this is a matter of definition. In Lettuce, we could pretend that the scoping rules allow the defining expression in a let binding to be evaluate first under previously defined let bindings. 

```
Therefore , the code fragment 
    
    let x = 10 in 
        let x = x + 10 in 
            let x = x + 10 in 
                x + 10 
```

is entirely equivalent to 

```
    let x0 = 10 in 
        let x1 = x0 + 10 in 
            let x2 = x1 + 10 in 
                x2 + 10
```

How about this one? 

```
    let y = 15 in 
        let x = ( let y = 10 in y + y ) in 
            y + x 
```

There are two places where y is being defined. So which bidning of y matters for y + x 

Answer is that this is entirely equal to 

```
    let y1 = 15 in 
        let x = ( let y2 = 10 in y2 + y2 ) in 
            y1 + x 
```

Thus the value of y is 15 when evaluating y + x but equals 10 when evaluating y + y. 

Scoping is not intuitive but very important understand

The same holds in scala. 
```
    val y = 15 
    val x = {
        val y = 10 
        y + y 
    }
    x + y 
```

In [6]:
val y = 15
val x = {
        val y = 10  // thsi overshadows the y = 15 delcaration 
        y + y
} 
print(s"y = $y, x + y = ${x + y}")

y = 15, x + y = 35

y: Int = 15
x: Int = 20

## Basic Types in Lettuce 

To begin with, we will allow varaibles in Lettuce be double precision numbers and Booleans. Later, of course, we will add more types to the langauge. This means we will allow the entire grammar of expressions and conditional expressinos we have previously defined into Lettuce. 

```
    let x = 25 in 
        let y = exp(x) in 
            if (z) 
                then y 
                else x 
```


Here x is defined as a double precision value 25 and y computes the exponential of x. Finally, z is a Boolean that computes true if x >= y and false otherwise. Finally, the entire expression evalautes to y if z is true otherwise x. 


## Function calls in Lettuce

We will allow definitons of functions and calls tot hem in Lettuce. All functions in Lettue can have just one argument to them. 

```
    let w = 3.1415 in 
    let f = function (x)
        let y = 2 * w - 5 in 
        let z = 2 * w * x in 
                y * sin(z) 
        in 
        w * w + f(1) 
```

Here we ahve defined a functino f using let bindings. 
```
    let <function name> = function (<formal parameter name>) 
                                       <body expression for function> 
                                 in 
                                     <body expression> 
```              

For the example above, the <function name> is f, the <formal paraemter name> is x, the <body expression for function is 

```
    let y = 2 * x - 5 in 
        let z = 2 * w * x in 
            y * sin(z) 
```

the other <body expression> is w * w + f(1)

It is important to understand scoping rules for a function call. First, the variable x inside the body of f refers to its formal parameter. What about the variable w inside of body of f? I tmust refer to the let w = 3.1415 in the that immediately preced its. If we made this chioce, it is called "static" scoping. IE variables insde a function are resolved at the time of defintiion of the function. A different choice can be made called dynamic scoping which would resolve  the variable not at the time when f is resolved but for when f is actually called. 

For instance, consider
```
    let w = 3.1415  in 
    let f = function (x) 
            x * sin(2 * w * x) 
          in 
    let w = 3.1415 / 2.0 in 
        w * w + f(1)

```

Under static scoping, the w inside the body of function f resolves to 3.1415, whereas the w at call to fw * w + f(1) resolves to 3.1415/2.0


However, under dynamic scoping, the value of w insdie the function f is bound to whatever value it takes when f is called. Thus, it would be bound to 3.1415 / 2.0 since that is what w is whne we call f(1). If you think of dynamic scoping as "counter intuitive", just think again. It is the scoping that python uses. 

```
# Here is apython program
w = 25 
def f(x): 
    return x + 1 
w = 50 
print(f(30)) # does this print 55 or 80

    python is dynamically typed so it would take on f(50))
```

What kind of scoping does scala have? Can we try? First of all, scala does not permit redefining a val insdie of the same scope. The program will raise a syntax error

```
    val w = 25 
    def f(x: Int): Int = x + w 
    val w = 50 
    val z = f(30) 
```

But we can try to runa progrma like this and conclude that scala does static scoping as well

In [2]:
val w = 10
def f(x: Int): Int = {print(s"From inside : w = $w"); x + w  }
val z = {
    val w = 20
    f(20)
}

From inside : w = 10

w: Int = 10
defined function f
z: Int = 30

## Currying Functions

Lettuce has just one argument functions. How do we do multiple argument functions in Lettuce? Simple, we can always use currying to define a function with multiple arguments. 

```
    let f = function (x) function(y) x + y in 
        f(10)(20)
```

This defines a function of two arguments f as first a function that takes in x and returns a function that takes in y and adds x + y. 

An equivalent way of writing this code would be. 

```
    let f = function (x) 
        let g = function (y) x + y in 
                g 
        in 
      let y1 = f(10) in 
          y1(20) 
```

currying is also supported in scala. 

In [7]:
def  f(x: Int) (y: Int) = x + y 

// partially applied function ? 
val v1:  Int => Int = f(10)

val v2 = v1(20)

val v3 = f(40)(40)

defined function f
v1: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2220/1478979317@4380a207
v2: Int = 30
v3: Int = 80

Currying is very useful in the following situation: 
    * Apply some operations to your inputs and 
    * call a user dfeined function on the result

In [5]:
def multiplyAndProcess(x: Int, y: Int) (f: Int => Int) = {
    f (x* y) 
}

val v1  = multiplyAndProcess(20,10) {x => 2 * x }

defined function multiplyAndProcess
v1: Int = 400

## Grammar for Lettuce

We are now ready to define a grammar for Lettuce.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Div (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Log (\mathbf{Expr}) \\
 & | & Exp (\mathbf{Expr}) \\
 & | & Sine (\mathbf{Expr}) \\
 & | & Cosine (\mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Not ( \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - identifier(expr)} \\[5pt]
\end{array}$$


It helps to know how programs written in concrete syntax translate into abstract syntax using the grammar. 


## Example 1
```
    let x = 10 + 15 in 
        let y = x >= 25 in 
            if (y) 
            then x 
            else x - 35 

will translate to
    
    Let("x", Plus(Const(10), Const(15)), 
        Let("y", Geq(Ident("x"), Const(25)), 
                    IfThenElse( Ident("y"), 
                                    Ident("x"), 
                                        Minus(Ident("x"), Const(35))
                                  )
                   )    
     )
```

## Example 2 

```
        let square = function (w) w * w in
            25 + square(25) 
            
will translate to 
    
        Let("square", FunDef("w", Mult(Ident("w"), Ident("w")) ), 
                  Plus(Const(25), FunCall(Ident("Square"), Const(25)))
```

In [21]:
sealed trait Program 
sealed trait Expr 

case class TopLevel(e: Expr) extends Program

// base expressions
case class Const(v: Double) extends Expr
case object True extends Expr
case object False extends Expr 
case class Ident(s: String) extends Expr 

// arithmetic expressions 
case class Plus(e1: Expr, e2: Expr) extends Expr 
case class Minus(e1: Expr, e2: Expr) extends Expr 
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Div(e1: Expr, e2: Expr) extends Expr 
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr 
case class Sine(e: Expr) extends Expr 
case class Cosine(e: Expr) extends Expr 

// Boolean expressions
case class Geq(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr 
case class Not(e: Expr) extends Expr 

// If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr 

// Let bindings , <symbol> <defining expression> <body expression its used in>
// after you  eval e1, you need to extend the environment for e2 to be evaluated 
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr 

// function definition 
case class FunDef(para: String, bodyExpr: Expr) extends Expr 

// Function call 
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr 


defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine
defined class Geq
defined class Eq
defined class And
defined class Or
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

In [10]:
val p1 = TopLevel(Let("x", Plus(Const(10), Const(15)), 
         Let("y", Geq(Ident("x"), Const(25)), 
                IfThenElse( Ident("y"), 
                               Ident("x"), 
                               Minus(Ident("x"), Const(35))
                           )
             )
    ))

p1: TopLevel = TopLevel(
  Let(
    "x",
    Plus(Const(10.0), Const(15.0)),
    Let(
      "y",
      Geq(Ident("x"), Const(25.0)),
      IfThenElse(Ident("y"), Ident("x"), Minus(Ident("x"), Const(35.0)))
    )
  )
)

In [11]:
val p2 = TopLevel(Let("square", FunDef("w", Mult(Ident("w"), Ident("w")) ), Plus(Const(25), FunCall(Ident("square"), Const(25)))))

p2: TopLevel = TopLevel(
  Let(
    "square",
    FunDef("w", Mult(Ident("w"), Ident("w"))),
    Plus(Const(25.0), FunCall(Ident("square"), Const(25.0)))
  )
)

## Well Formed vs. Ill Formed Programs 

You may already note that the abstract syntax allows us to wrtie all kinds of program sthat should be clearly illegal. Here are som examples. 

    let x = y + z in y * z 
    
y an d z are undeclared a tht the itme that we are binding y + z to x , same problem when we evaluate y * z 

```
    let y = 10 in 
    let z = 15 in 
    let x = y + (z = 25) in 
        false
```
we are trying to add a number y and a boolean expression z >= 25 what is the meaning? 

```
    let y = 15 in 
    let z = 25 + function (w) w * w in 
        y(31) 
```
Wow! where to begin? first of all, we are adding 25 to a function definition function (w) w * w and we are calling y(31) when y is not even a function. 

Programs like these are really confusing. Do we even allow them? Why can't we just forbid them in the abstract syntax tree? 

__Fact:__ abstract syntax trees are built from the Context Free Grammars. Unfortunately, due to teh limitations arising from context freedom (ie LHS of each rule is just a single nontemrinal), we cannot really enforce rules like. 

    * Variables should be declared before use 
    * Boolean expressions cannot be added to integer expressions
    * function calls can only be made over proper functions 
    
Programming langauges generally have different enforcement mechanisms: 
    * check if a program is well formed while parsing the program
    * check types of expressions while parsing the program (static type checking) 
    * Check types of expressions while interpreting/evaluating the program (runtime typechecking) 
    * A combination of above. 
    
Let us use scala as an example. Certain issues are cuaght at compile time     

In [11]:
// Adding a number to a boolean, scala will catch it right away 
def f(x: Int): Int = { x + (x >= 233) }

cmd11.sc:1: overloaded method value + with alternatives:
  (x: Int)Int <and>
  (x: Char)Int <and>
  (x: Short)Int <and>
  (x: Byte)Int
 cannot be applied to (Boolean)
def f(x: Int): Int = { x + (x >= 233) }
                         ^Compilation Failed

: 

In [ ]:
// Comparing a string to a number? 
def comp(x: String) : Boolean = x >= 233 

In [11]:
// Trying to call a number? 
def nonsence(x: Int) : Int = x(355) 

cmd11.sc:1: Int does not take parameters
def nonsence(x: Int) : Int = x(355) 
                              ^Compilation Failed

: 

In [12]:
// this error is caught a runtime
def incompleteCase(x: (Int, Int, Int)): Int = x match {
    case (i, j, k) if i == 0 => j - k
    case (i, j, k) if j > 0 => i - k
    case (i, j, k) if k > 0 => j - i
    // Clearly more cases are needed here. 
    // Will scala complain when compiling this?
}

defined function incompleteCase

In that same vein, we can catch errors in Letuce programs at compile time. 
    * Variables should be declared before use (we can catch thi susing a specifal decalre before use check) 
    * Boolean expressions cannot be added to integer expressions (we will catch this when we do type checking) 
    * Function calls can only be made over proper functions (we will catch this during type checking) 

# Example: Checking whether variables are declared before usage. 

Let us define inference rules that will help us check if an expression is checked before use. To do so, we will define a set of currently declared variables that are in scope called S. 

we will define inference rules for judgements of the form: WellFormed(e,S) that says that expression e is wellFormed under the set of declared in scope variables S 

Constants are well formed under any set S. 

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, S) \\
\end{array} \text{(const-rule)} $$

The rules for `True` and `False` are similar.
Identifiers are well formed only if they belong to $S$:

$$\begin{array}{c}
x \in S \\
\hline
WellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{(ident-rule)} $$

The rules for binary operator `Plus, Minus, Mult, Div,  Geq, Eq, And, Or` are all very similar.

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Div}, \texttt{Geq}, \texttt{Eq}, \texttt{And}, \texttt{Or} \} \\
\hline
WellFormed(\texttt{T(e1, e2)}, S) \\
\end{array} \text{(well-formed-binary-op)} $$

Likewise, for unary operators 

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; T \in \{ \texttt{Log}, \texttt{Exp}, \texttt{Sine}, \texttt{Cosine}, \texttt{Not}, \texttt{Eq} \} \\
\hline
WellFormed(\texttt{T(e1)}, S) \\
\end{array} \text{(well-formed-unary-op)} $$

The main rule is for let bindings

$$\begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S \cup \{ x\} ) \\
\hline
WellFormed(\texttt{Let(x, e1, e2)}, S) \\
\end{array} \text{(let-rule)} $$

Suppose `e1` is well formed under $S$ and `e2` is well formed under $S$ with the identifier `x` added,
then `let x = e1 in e2` is well formed under $S$, as well. 
- This is confusing when you first see it, but repeat the sentence above and match it with the rule, 
- Notice that the defining expression must be well formed under the original S but the body expression is allowed to be well formed under $S \cup \{ x\} $.
- Once you actualy get it, you can appreciate how delightfully _simple_ it is!

We can also add the rules for function definition:

$$\begin{array}{c}
WellFormed(\texttt{e}, S \cup \{x\}) \\
\hline
WellFormed(\texttt{FunDef(x, e)}, S ) \\
\end{array} \; \text{(fundef-rule)} $$

Finally, function calls are very simple:

$$ \begin{array}{c}
\texttt{f} \in S,\;\;\;\; WellFormed(\texttt{e}, S) \\
\hline
WellFormed(\texttt{f(e)}, S ) \\
\end{array} \text{(well-formed-funcall)} $$

The name of the function `f` better be defined in the scope $S$ and the argument better be OK. Then
we can say that the function call is.

With these rules, we can bravely write our first static check for Lettuce to catch programs that declare variables before use.

In [14]:
/* Remember the whole point of this is to just make sure that our program is typed 
    correctly */ 
def isWellFormedExpression(e: Expr, S: Set[String]): Boolean = e match {
    case Const(_) | True | False  => true
    
    // we are checking here if an string 'x' is defined in our set S (our environment) 
    case Ident(x) => {
        if (S contains x) 
        { true } 
        else 
        { println(s"Error: undeclared identifier: $x");
          false }
        }
    
    // binary operations are just a matter of checking if both are well formed 
    case Plus(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Minus(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Mult(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Div(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Geq(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Eq(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case And(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Or(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Log(e) => isWellFormedExpression(e, S) 
    case Exp(e) => isWellFormedExpression(e, S) 
    case Sine(e) => isWellFormedExpression(e, S) 
    case Cosine(e) => isWellFormedExpression(e, S) 
    case Not(e) => isWellFormedExpression(e, S) 
    
    case IfThenElse(e1, e2, e3) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S) && isWellFormedExpression(e3, S)
    
    case Let(x, e1, e2) => isWellFormedExpression(e1, S) && { 
        val S1 = S + x  // need to extend the map! so we can resolve e2 if its well formed
        isWellFormedExpression(e2, S1)
    }
    
    // <formal parameter> , <function body expression> 
    case FunDef(x, e) => isWellFormedExpression(e, S+x) // what is x initially though? 
    
    // <function name> <argument we are passing> 
    case FunCall(f, e) => isWellFormedExpression(f, S) && isWellFormedExpression(e, S) 
}


def isWellFormedProgram(p: Program)  = p match {
    case TopLevel(e) => isWellFormedExpression(e, Set())
}

defined function isWellFormedExpression
defined function isWellFormedProgram

In [15]:
isWellFormedProgram(p1)

res14: Boolean = true

In [16]:
isWellFormedProgram(p2)

res15: Boolean = true

In [17]:
val p3 = Let("x", Plus(Ident("y"), Ident("z")), Mult(Ident("x"), Ident("y"))) 

p3: Let = Let("x", Plus(Ident("y"), Ident("z")), Mult(Ident("x"), Ident("y")))

In [18]:
isWellFormedProgram(TopLevel(p3))

Error: undeclared identifier: y


res17: Boolean = false

In [19]:
val p4 = Let("x", Ident("x"), Mult(Ident("x"), Ident("y"))) 

p4: Let = Let("x", Ident("x"), Mult(Ident("x"), Ident("y")))

In [20]:
isWellFormedProgram(TopLevel(p4))

Error: undeclared identifier: x


res19: Boolean = false

## Writing an Eval for Lettuce (Without Functions) . 

Let us write an eval function for Lettuce. We can work with the existing eval function that already defined for expressions, right? 

To avoid tackling to much in one swoop, let us ignore function definitions and function calls for now

Recall what $\sigma \models \texttt{e} \Downarrow v$ meant? 

Well, let's use less fancy notation and convey exactly the same sentiment using the notation
$$eval(\texttt{e}, \sigma) = v $$ 

- $\sigma$ refers to an environment that maps names of identifiers to their values.
- $\text{domain}(\sigma)$ refers to the domain of $\sigma$.
- $\phi$ will refer to the empty environment in which no identifier is defined.
- If $\sigma$ is an environment, then $\sigma[x \mapsto v]$ is a new environment in which the identifier $x$ is mapped to the value $v$.


What are the values we need to have: 

    * Real values belonging to the set R (or Double precision numbers in scala). We discuessed in class what the differences would be. 
    * Boolean values true, false to the set B 
    * the special value error to the set ERR 
    
The rules are mostly the same, save for a change in notation from $\sigma \models \texttt{e} \Downarrow v$ to 
$eval(\texttt{e}, \sigma) = v$.

$$\begin{array}{c}
\\
\hline
eval(\texttt{Const(v)}, \sigma) = v \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
x \in \text{domain}(\sigma) \\
\hline
eval(\texttt{Ident(x)}, \sigma) = \sigma(\texttt{x}) \\
\end{array} \text{(ident-ok-rule)}\ \;\;\; \begin{array}{c}
x \not\in \text{domain}(\sigma) \\
\hline
eval(\texttt{Ident(x)}, \sigma) = \mathbf{error} \\
\end{array} \text{(ident-nok-rule)} $$


Let us read the (ident-ok-rule) aloud. It says that if a _variable x belongs to the domain of $\sigma$_ then
_the expression `Ident(x)` evaluates to $\sigma(x)$_. Makes sense?
<font color="red"> Exercise: read the (ident-nok-rule) aloud in your own words. </font> 

<font color="red"> Q: </font> If we checked that the expression is well-formed, then will we need (ident-nok-rule)?

Rules for `Plus, Minus, Mult`. 

$$\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; \; eval(\texttt{e2}, \sigma) = v_2,\ \ v_1 \in \mathbb{R},\ \ v_2 \in \mathbb{R}, \; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = f_T(v_1, v_2) \\
\end{array} \text{(arith-binop-ok-rule)}$$

Note that the rule refers to $f_T$ for the operator $T$. 
Define these as $f_{Plus}(x,y) = x + y$, $f_{Mult} = x * y$ and $f_{Minus} (x, y) = x - y$.

The `Div` operator needs special handling in exactly the same way we showed in our previous lectures.
<font color="red"> Write down the rules for `Div` as an exercise </font>


We can now handle the case when subexpressions of arithmetic expressions yield a non real value (these can be Booleans or even error).

$$\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; \ v_1 \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult, Div} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-binop-type-mismatch-rule-1)}\;\;
\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; eval(\texttt{e2}, \sigma) = v_2,\; \ v_1 \in \mathbb{R},\; \; v_2 \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult, Div} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-binop-type-mismatch-rule-2)}
$$

Unary arithmetic operators are also handled in the same way as in the previous lectures.
$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R} \; \; \texttt{T} \in \{ \texttt{Exp, Sine, Cosine} \}  \\
\hline
eval(\texttt{T(e)}, \sigma) = f_T(v) \\
\end{array} \text{(arith-unop-ok-rule)}$$

Wherein $f_{Exp}(x) = e^x,\ f_{Sine}(x) = \sin(x), f_{Cosine}(x) = \cos(x)$.
Log needs special rule to deal with argument being positive vs. non-positive.

$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R},\ v > 0  \\
\hline
eval(\texttt{Log(e)}, \sigma) = \log(v) \\
\end{array} \text{(log-ok-rule)}\;\;\;
\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R},\ v \leq 0  \\
\hline
eval(\texttt{Log(e)}, \sigma) = \mathbf{error} \\
\end{array} \text{(log-nok-rule)}$$

$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\; \ v \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Log, Sine, Cosine, Exp} \}  \\
\hline
eval(\texttt{T(e)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-unop-type-mismatch-rule)}$$

#### Rules for Boolean Operators

$$\begin{array}{c} 
\\
\hline
eval(\texttt{True}, \sigma) = true \\
\end{array} \text{(true rule)} \;\;\;\;
\begin{array}{c} 
\\
\hline
eval(\texttt{False}, \sigma) = false \\
\end{array}\text{(false rule)}
$$

`And` and `Or` can be short circuited. We will write the rules for `And` and note that
the rules of `Or` and `Not` are similar.

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = false\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = false\\
\end{array} \text{(and-arg-1-ok-rule)} \;\;\;\;
\begin{array}{c} 
eval(\texttt{e1}, \sigma) = v_1,\ v_1 \not\in \mathbb{B} \\
\hline
eval(\texttt{And}(e1, e2), \sigma) = \mathbf{error}\\
\end{array}\text{(and-arg-1-nok-rule)}
$$

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = true\;\; eval(\texttt{e2}, \sigma) = v_2,\ \;\; v_2 \in \mathbb{B}\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = v_2\\
\end{array} \text{(and-arg-2-ok-rule)} \;\;\;\;
\begin{array}{c} 
eval(\texttt{e1}, \sigma) = true,\ eval(\texttt{e2}, \sigma) = v_2,\  \;\; v_2 \not\in \mathbb{B} \\
\hline
eval(\texttt{And}(e1, e2), \sigma) = \mathbf{error}\\
\end{array}\text{(and-arg-2-nok-rule)}
$$

<font color="red"> As an exercise write rules for  `Geq, Eq` </font> . They are going to be very similar to those written thus far but without short circuit semantics for regular case and with short circuit for errors.

#### Rule for Let Binding

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = v_1,\ v_1 \not= \mathbf{error}\;\; eval(\texttt{e2},   \color{red}{\sigma[x \mapsto v_1]}) = v_2,\ \;\; v_2 \not= \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = v_2\\
\end{array} \text{(let-binding-ok)} $$

The most important part of the rule above is notice that $\texttt{e2}$ is being evaluated under
$\color{red}{\sigma[ x \mapsto v_1]}$, which is the environment $\sigma$ extended with $x$ bound to $v_1$.

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) =  \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = \mathbf{error}\\
\end{array} \text{(let-binding-nok-1)}
\begin{array}{c} 
eval(\texttt{e1}, \sigma) =  v_1,\; v_1 \not= \mathbf{error}\; eval(\texttt{e2}, \sigma[x \mapsto v_1]) =  \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = \mathbf{error}\\
\end{array} \text{(let-binding-nok-2)}$$


Given these rules, the final rule is for `TopLevel`. We recall that $\phi$ denotes the empty environment.

$$\begin{array}{c}
eval(\texttt{e}, \phi) = v \\
\hline
eval(\texttt{TopLevel(e)}, \phi) = v \\
\end{array} \text{(toplevel-eval)} $$   


## Implementing the Interpeter 
First let us implement what we will need as part of the value class. 

In [22]:
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case object ErrorValue extends Value

/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}


defined trait Value
defined class NumValue
defined class BoolValue
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean

In [25]:
def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
        case True => BoolValue(true)
    
        case False => BoolValue(false)
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
        case Div(e1, e2) => applyArith2(e1, e2) {
            case (_, 0.0) => throw new IllegalArgumentException(s"Divide by zero error in divisor ${e2}")
            case (v1, v2) => v1/v2
        }
    
        case Log(e1) => applyArith1(e1) {
            case v if v > 0.0 => math.log(v)
            case v => throw new IllegalArgumentException(s"Log of a negative number ${e} evaluates to ${v}!")
        }
    
        case Exp(e1) => applyArith1(e1)(math.exp)
    
        case Sine(e1) => applyArith1(e1)(math.sin)
    
        case Cosine(e1) => applyArith1(e1)(math.cos)
    
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        case And(e1, e2) => { /* Short circuit eval of And */
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(false) => BoolValue(false)
                case BoolValue(true) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"And of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"And of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Or(e1, e2) => { /* Short circuit eval of OR*/
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(true) => BoolValue(true)
                case BoolValue(false) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"Or of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"Or of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Not(e1) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(b) => BoolValue(!b)
                case _ => throw new IllegalArgumentException(s"Not of a non-boolean expr: ${e} which evaluated to ${v}")
            }
        }
    
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case _:FunDef => throw new IllegalArgumentException("Function definitions not yet handled in this interpreter.")
    
        case _:FunCall => throw new IllegalArgumentException("Function calls not yet handled in this interpreter.")
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}

defined function evalExpr
defined function evalProgram

In [25]:
print(p1)
evalProgram(p1)

cmd25.sc:2: type mismatch;
 found   : ammonite.$sess.cmd9.wrapper.cmd8.TopLevel
 required: ammonite.$sess.cmd24.wrapper.cmd20.Program
val res25_1 = evalProgram(p1)
                          ^Compilation Failed

: 

In [25]:
print(p4)
evalProgram(TopLevel(p4))

cmd25.sc:2: type mismatch;
 found   : ammonite.$sess.cmd18.wrapper.cmd8.Let
 required: cmd25.this.cmd20.Expr
val res25_1 = evalProgram(TopLevel(p4))
                                   ^Compilation Failed

: 

In [26]:
val p5 = TopLevel(Let("x", Const(3.0), Mult(Ident("x"), Ident("x"))))

p5: TopLevel = TopLevel(Let("x", Const(3.0), Mult(Ident("x"), Ident("x"))))

In [27]:
evalProgram(p5)

res26: Value = NumValue(9.0)

In [28]:
val p6 = TopLevel(Let("x", Const(3.0), Geq(Mult(Ident("x"), Ident("x")), Ident("x"))))

p6: TopLevel = TopLevel(
  Let("x", Const(3.0), Geq(Mult(Ident("x"), Ident("x")), Ident("x")))
)

In [29]:
evalProgram(p6)

res28: Value = BoolValue(true)

# End of pt1 Lettuce. 

# Function Calls in Lettuce 

Previously, we did not handle function calls in our fledging Lettuce interpreter. Now we will add support for function calls. Before we do so, let us recall the full grammar. 


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Div (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Log (\mathbf{Expr}) \\
 & | & Exp (\mathbf{Expr}) \\
 & | & Sine (\mathbf{Expr}) \\
 & | & Cosine (\mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Not ( \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\[5pt]
\end{array}$$


Here is the scala definition again.

In [2]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case object True extends Expr // Expr -> True
case object False extends Expr // Expr -> False
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)
case class Div(e1: Expr, e2: Expr) extends Expr // Expr -> Mult(Expr, Expr)
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings, let a string s be defined by _ in _ , scoping stuff
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition, we name the function in the Let
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call  , this is just used for calling 
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine
defined class Geq
defined class Eq
defined class And
defined class Or
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

## Example 1 
```
    let square = function(x) 
        x * x in 
            square(10)

```

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$


by definition, when we evaluate a FunDef() we set it equal to the Closure(parameters, function body, and the environment that is currenty at hand. we do not extend the environment until we actually try to evaluate the funcitno body. 

In [4]:
/* 
    Have to define the function, in the abstract syntax, then call 
    the function with a FunCall 
    
        Let square = function (x) 
                        x * x 
                            in 
                                FunCall("square", 10)
*/
val p1 = TopLevel(
    Let("square",                            // let square = 
       FunDef("x", Mult(Ident("x"), Ident("x"))),  // function(x) x*x
        FunCall(Ident("square"), Const(10))   // in square(10) 
       )
)

/*
    Let("square", FunDef("x"), Mult(Ident("x"), Ident("x"))), FunCall(Ident("square"), Const(10.0))
    
*/

p1: TopLevel = TopLevel(
  Let(
    "square",
    FunDef("x", Mult(Ident("x"), Ident("x"))),
    FunCall(Ident("square"), Const(10.0))
  )
)

## Example 2 
```
    let x = 10 in 
    let y = 15 in 
    let sql = function(x)
                function(y) 
                    x + y * y
              in 
            sql(x)(y) 
```

__question:__ Map the differant usages of x, y in the code above to the apprirate defintions? 

This program evalautes to 235

In [5]:
val x = Ident("x")
val y = Ident("y")
val fdef_inner = FunDef("y", Plus(x, Mult(y, y)))
val fdef_outer = FunDef("x", fdef_inner)
val call_expr = FunCall(FunCall(Ident("sq1"), x), y)
val sq1_call = Let("sq1", fdef_outer, call_expr)
val lety = Let("y", Const(15), sq1_call)
val letx = Let("x", Const(10), lety)
val p2 = TopLevel(letx)

x: Ident = Ident("x")
y: Ident = Ident("y")
fdef_inner: FunDef = FunDef("y", Plus(Ident("x"), Mult(Ident("y"), Ident("y"))))
fdef_outer: FunDef = FunDef(
  "x",
  FunDef("y", Plus(Ident("x"), Mult(Ident("y"), Ident("y"))))
)
call_expr: FunCall = FunCall(FunCall(Ident("sq1"), Ident("x")), Ident("y"))
sq1_call: Let = Let(
  "sq1",
  FunDef("x", FunDef("y", Plus(Ident("x"), Mult(Ident("y"), Ident("y"))))),
  FunCall(FunCall(Ident("sq1"), Ident("x")), Ident("y"))
)
lety: Let = Let(
  "y",
  Const(15.0),
  Let(
    "sq1",
    FunDef("x", FunDef("y", Plus(Ident("x"), Mult(Ident("y"), Ident("y"))))),
    FunCall(FunCall(Ident("sq1"), Ident("x")), Ident("y"))
  )
)
letx: Let = Let(
  "x",
  Const(10.0),
  Let(
    "y",
    Const(15.0),
    Let(
      "sq1",
      FunDef("x", FunDef("y", Plus(Ident("x"), Mult(Ident("y"), Ident("y"))))),
      FunCall(FunCall(Ident("sq1"), Ident("x")), Ident("y"))
    )
  )
)
p2: TopLevel = TopLevel(
  Let(
    "x",
    Const(10.0),
    Let(
      "y",
      Con

## Example 3 

```
let h = function(x) 
        log(z)
      in 
let g = function(y)
        y/2.0 + h(y * 1.5)
      in 
let f = function (x) 
    1.0/x + g(x) 
in 
    f(3.1415) 

```

This should evaluate to 3.4392347...

In [6]:
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")

val fDef = FunDef("x", Plus(Div(Const(1.0), x), FunCall(Ident("g"), x)) )
val gDef = FunDef("y", Plus(Div(y, Const(2.0)), FunCall(Ident("h"), Mult(y, Const(1.5)))))
val hDef = FunDef("z", Log(z))

val letf = Let("f", fDef, FunCall(Ident("f"), Const(3.1415)))
val letg = Let("g", gDef, letf)
val leth = Let("h", hDef, letg)

val p3 = TopLevel(leth)

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
fDef: FunDef = FunDef(
  "x",
  Plus(Div(Const(1.0), Ident("x")), FunCall(Ident("g"), Ident("x")))
)
gDef: FunDef = FunDef(
  "y",
  Plus(
    Div(Ident("y"), Const(2.0)),
    FunCall(Ident("h"), Mult(Ident("y"), Const(1.5)))
  )
)
hDef: FunDef = FunDef("z", Log(Ident("z")))
letf: Let = Let(
  "f",
  FunDef(
    "x",
    Plus(Div(Const(1.0), Ident("x")), FunCall(Ident("g"), Ident("x")))
  ),
  FunCall(Ident("f"), Const(3.1415))
)
letg: Let = Let(
  "g",
  FunDef(
    "y",
    Plus(
      Div(Ident("y"), Const(2.0)),
      FunCall(Ident("h"), Mult(Ident("y"), Const(1.5)))
    )
  ),
  Let(
    "f",
    FunDef(
      "x",
      Plus(Div(Const(1.0), Ident("x")), FunCall(Ident("g"), Ident("x")))
    ),
    FunCall(Ident("f"), Const(3.1415))
  )
)
leth: Let = Let(
  "h",
  FunDef("z", Log(Ident("z"))),
  Let(
    "g",
    FunDef(
      "y",
      Plus(
        Div(Ident("y"), Const(2.0)),
        FunCall(Ident("h"), Mult(Iden

## Example 4 (bad) 

We can ask what happens if we define a recursive function like so: 
```
    let f = function (x) 
            if ( 0 >= x )
                1
            else
                (x - 1) * f(x - 1) 
            in 
            f(10) 
```

We can predict: note that the let expression
    let f = exprA in expr B 
    
we note that f is __not__ in scope for exprA. Therefore the reursive call f(x-1) , we will get an error as we see. 

__ we will modify this situation to allow recursive definitions coming next. But for now, this sort of a call will lead to an error __

In [8]:
val x = Ident("x")
val compX = Geq(Const(0), x) 
val recExpr = Mult(Minus(x, Const(1.0)), FunCall(Ident("f"), Minus(x, Const(1.0))))
val f_defn = FunDef("x", IfThenElse(compX, Const(1.0), recExpr))
val letf = Let("f", f_defn, FunCall(Ident("f"), Const(10.0)))
val p4 = TopLevel(letf) 

x: Ident = Ident("x")
compX: Geq = Geq(Const(0.0), Ident("x"))
recExpr: Mult = Mult(
  Minus(Ident("x"), Const(1.0)),
  FunCall(Ident("f"), Minus(Ident("x"), Const(1.0)))
)
f_defn: FunDef = FunDef(
  "x",
  IfThenElse(
    Geq(Const(0.0), Ident("x")),
    Const(1.0),
    Mult(
      Minus(Ident("x"), Const(1.0)),
      FunCall(Ident("f"), Minus(Ident("x"), Const(1.0)))
    )
  )
)
letf: Let = Let(
  "f",
  FunDef(
    "x",
    IfThenElse(
      Geq(Const(0.0), Ident("x")),
      Const(1.0),
      Mult(
        Minus(Ident("x"), Const(1.0)),
        FunCall(Ident("f"), Minus(Ident("x"), Const(1.0)))
      )
    )
  ),
  FunCall(Ident("f"), Const(10.0))
)
p4: TopLevel = TopLevel(
  Let(
    "f",
    FunDef(
      "x",
      IfThenElse(
        Geq(Const(0.0), Ident("x")),
        Const(1.0),
        Mult(
          Minus(Ident("x"), Const(1.0)),
          FunCall(Ident("f"), Minus(Ident("x"), Const(1.0)))
        )
      )
    ),
    FunCall(Ident("f"), Const(10.0))
  )
)

# Scoping for Function Calls: Static vs. Dynamic

It is important to understand how function calls capture scopes by considering a few examples both in lettuce and scala


### Example 1 
```
    let x = 10 in 
        let f = function(y) y * x in 
            let x = 15 in 
                f(10)
                
Or equivalently in scala
    
    val x = 10
    val f = (y:Int) => (y*x) 
    {
        val x = 15 
          f(10)
    }
```

In both cases our cod ehas function f that multiplies its parameter y by x. But precisely which of the x should the function use? 

    * let x = 10 in the first that is in scop when the function was first defined?
    * Or let x = 15 in the third line that is in scope when the function is actually being called? 

In [9]:
// As you will see here, scala uses the x when defined => STATIC 
val x = 10 
val f = (y: Int) => (y * x)
{
    val x = 15 
    println("f(10) = " + f(10))  // 10 * 10 
}

f(10) = 100


x: Int = 10
f: Int => Int = ammonite.$sess.cmd8$Helper$$Lambda$2773/1913205951@3cf62e48

We can conclude that scala uses the definition of x that was in scope __when the function was defined__. This kind of scoping is known as __static__ scoping. As opposed to dynamic scoping, in which case the value of x used is what is in the scope when the function is called. 



## Why static scoping? 
Static scoping enables many useful programming idioms, we mention a few. 

__A) Partial application of functions__ 
Static scoping allows us to define functions some of whose arguments are already bound at defintion time. 

In [13]:
// MatchAgainstList takes in a reference_lst and returns a function
// that itself takes in a list 
// why is the syntax so wierd
def matchAgainstList(reference_lst: List[Int]) (lstB: List[Int]) = {
    def belongsToRefList(x: Int) = {
        reference_lst.contains(x) 
    }
    lstB.exists(belongsToRefList) 
}

val refList = List(1,2,3,4,5,6,7,8,9,10) 
val boundMatchFun: (List[Int] => Boolean) = matchAgainstList(refList)(_)
{
    val refList = List(11,12,13,14) 
    boundMatchFun(List(5,6,7,8))
}


defined function matchAgainstList
refList: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
boundMatchFun: List[Int] => Boolean = ammonite.$sess.cmd12$Helper$$Lambda$2938/708670874@2b316981
res12_3: Boolean = true

__B) Callbacks__ 
In the same vein as before, we can define callback to handle events using partially applied function to bind extra information into the callback function 

In [17]:
abstract class Event
case class KeyPress(keyID: Int) extends Event
case class MouseClick(x: Int, y: Int) extends Event
case object PrintStatistics extends Event

def keyPressEventHandlerFactory(): (Event => Unit) = {
    // Make a callback function and return it.
    var keysPressedSoFar: Set[Int] = Set() // Mutable state
    def keyPressEventCallBack(event: Event): Unit = {
        event match {
            case KeyPress(keyID) => { 
                keysPressedSoFar = keysPressedSoFar ++ Set(keyID)
            }
            case PrintStatistics => {
                println("Keys Pressed so far are: " + keysPressedSoFar)
            }
            case _ => ()
        }
    }
    keyPressEventCallBack
}

def handleEventStream() = {
    val eventsToTest = List[Event](KeyPress(1), KeyPress(2), MouseClick(10,20), KeyPress(10), KeyPress(-1), PrintStatistics)
    val keyPressEventCallBack = keyPressEventHandlerFactory()
    eventsToTest.foreach (keyPressEventCallBack)
}

handleEventStream()

Keys Pressed so far are: Set(1, 2, 10, -1)


defined class Event
defined class KeyPress
defined class MouseClick
defined object PrintStatistics
defined function keyPressEventHandlerFactory
defined function handleEventStream

## Implementing Dynamic Scoping

Turns out implementing dynamic scoping is conceptually easy. We could just "inline" the body of the function and substitute for the calee argument. 

"inline" means that everwhere f is called, simply replace it by y*x and substitute the arguemnt of the call in place of y

```
           STATICALLY SCOPED
    let x = 10 in 
        let f = function(y) y * x in 
            let x = 15 in 
                f(10)
                
                
   ... after inlining ... 
           DYNAMICALLY SCOPED
   let x = 10 in 
       let f = function(y) y * x in 
           let x = 15 in 
               (10 * x) 
               
             pretty much all you did was execute what
             the function was going to do, but you manually
            made sure you wanted to have which variables you wanted 
             to be referenced. 
```

### Example 2 
```
    let y = 15 in 
        let x = x * y in 
            let function = function(z) z * y + x in 
                let x = y in 
                    f(10) + f(15) 
```

Evaluate the program above under both static and dynamic scoping. Under dynamic scoping note that the program can be seen as the equivalent prgram below: 

```
    let y = 15 in 
        let x = x * y 
            let f = function (z) z * y + x in 
                let x = y in 
                    (10 * y + x) + (15 * y + x) 
```

Dynamic scoping can be troublesome in many ways. It is hard for the programmar to reason about dynamic scoping since whoever implements a function f must be careful about the calling environment of the function which defines not just the formal arguments but also the identifiers in the function body that may reference. 

Under dynamic scoping simple change of vairable names can change the meaning of the function in unexpected ways. 

```
    let y = 15 in 
        let x = x* y in 
            let f = function (z) z * y + x in 
                let xNew = y in ## User changed the name of x to xNew
                    f(10) + f(15) 
```

we will now explain how static scoping is implemented using closures. 

##  Evaluating Lettuce with Function Calls
We are now ready to start evaluating Lettuce with function calls

Thus far, we have values R (for reals) nad B (for booleans), __error__ (special error). Now we will augment it with a new type of value for functions that will be termed a 'closure'


### Closure
A closure is defined as a combination of two things
    * A function definiton function(x) expr where x is the formal parameter to the call and expr is the body function of the call
    * an environment, env, that will define all variables occurring freely in the expr but not x. 
    
    
We will write a closure as 

    Closure(x, e, env) that represents a closure involving the
    
    function defintion
        function (x) e with environment, env
    
    Let us see some examples of closures. 
    

### Example 1

Consider a function definition

~~~
function(y) x + y * y
~~~

`y` is the formal parameter but `x` is an identifier that is not a formal parameter of the function.
We say that it occurs freely in the definition of the function.

An example of a closure will be

$\text{Closure}( \texttt{y},\ \underset{\text{Func. Expr.}}{ \underbrace{\texttt{x + y * y }}}, \underset{\text{Env.}\ \sigma}{\underbrace{\{ x \mapsto 10 \}}}  ) $.


__Note:__ All that is missing is a value for the formal parameter `y`. Everything else is in place to evaluate the
function. The moment `y` is available, we will have everything needed to execute the function body `x + y * y`.

### Example 2

Consider the function definition

~~~
   function(x) function(y) x + y - 2 * z
~~~

`x` is the formal parameter of the function and the expression is `function (y) x + y - 2 * z `.
Note that `z` occurs freely in the expression. The identifier `y` is not free since it is bound
to the formal parameter of the inner function.

A closure needs to specify a value for `z`. 

$\text{Closure}(x,\ (\texttt{function(y) x + y - 2 * z}),\  \{ z \mapsto 100 \} )$


## Operational Semantics with Function Call

Recall that $\text{eval}(\texttt{e}, \sigma) = v$ states that evaluating the expression `e` under the
environment $\sigma$ yields the value $v$.

Now our values can be of the form:
- Real numbers: $\mathbb{R}$,
- Booleans: $\mathbb{B}$,
- Closures: $\mathbb{C}$ of the form $\text{Closure}(x, \texttt{e}, \sigma)$,
- Error $\mathbf{error}$.

We will write a rule for handling function definitions for static typing. 
Let us recall what is static typing? It captures the environment at the time 
a function is defined. This is exactly what the closure does.

$$ \begin{array}{c}
\\
\hline
\text{eval}(\texttt{FuncDef}(x, e), \sigma) = \text{Closure}(x, \texttt{e}, \sigma) \\
\end{array} \text{(func-def-ok)}$$

All it remains to specify what happens when we call a function.

This is the really important rule to understand: the rest will be easy if this particular idea sinks in. 
So pay very careful attention:

$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = v_2,\ v_2 \not= \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \texttt{eval}( \color{red}{\texttt{e}}, \color{green}{\pi} [ \color{blue}{p} \mapsto v_2 ] )\\
\end{array} \text{(funcall-ok)}
$$


The rule concerns itself with evaluating a function call `FunCall(fun_expr, arg_expr)` of a call `fun_expr(arg_expr)` where
- `fun_expr` represents the function we are going to call.
- `arg_expr` represents the argument to the call.

First, if `fun_expr` is a function, then it better evaluate to a _closure_.
- This is expressed by the condition $\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi})$ on the top of the bar. This closure has a formal param $p$, 
body $\texttt{e}$ and the environment that was saved at function define time is $\pi$.

Next, we evaluate the argument to the funciton call `arg_expr`. Let this evaluate to $v_2$ where $v_2$ is not an __error__ value.

Finally, we are ready to call the closure. Note that with the arrival of $v_2$ from previous step, we have
everything needed to evaluate a function call. So we can extend the environment $\pi$ by binding 
$p$ the formal parameter to $v_2$. With that done, everything is ready to evaluate the body of the closure
$\texttt{e}$.

We need to add error rules. If the function expression is not a closure:
$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) \not \in \mathbb{C}\\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \mathbf{error}\\
\end{array} \text{(funcall-nok-1)}
$$

If the argument to the call leads to error:
$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{x}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \mathbf{error}\\
\end{array} \text{(funcall-nok-2)}
$$


In [19]:
/* 1. Defining the values */ 
sealed trait Value
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value

/* Let us add closure to the set of values */ 
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value
case object ErrorValue extends Value



/* 2. Operators on values */ 

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure

In [20]:
def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
        case True => BoolValue(true)
    
        case False => BoolValue(false)
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
        case Div(e1, e2) => applyArith2(e1, e2) {
            case (_, 0.0) => throw new IllegalArgumentException(s"Divide by zero error in divisor ${e2}")
            case (v1, v2) => v1/v2
        }
    
        case Log(e1) => applyArith1(e1) {
            case v if v > 0.0 => math.log(v)
            case v => throw new IllegalArgumentException(s"Log of a negative number ${e} evaluates to ${v}!")
        }
    
        case Exp(e1) => applyArith1(e1)(math.exp)
    
        case Sine(e1) => applyArith1(e1)(math.sin)
    
        case Cosine(e1) => applyArith1(e1)(math.cos)
    
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        case And(e1, e2) => { /* Short circuit eval of And */
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(false) => BoolValue(false)
                case BoolValue(true) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"And of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"And of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Or(e1, e2) => { /* Short circuit eval of OR*/
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(true) => BoolValue(true)
                case BoolValue(false) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"Or of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"Or of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Not(e1) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(b) => BoolValue(!b)
                case _ => throw new IllegalArgumentException(s"Not of a non-boolean expr: ${e} which evaluated to ${v}")
            }
        }
    
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}

defined function evalExpr
defined function evalProgram

In [21]:
val v1 = evalProgram(p1)

v1: Value = NumValue(100.0)

In [22]:
val v2 = evalProgram(p2)

v2: Value = NumValue(235.0)

In [23]:
val v3 = evalProgram(p3)

v3: Value = NumValue(3.4392347752010837)

In [24]:
val v4 = evalProgram(p4) // Will say that f is undefined identifier since we have let f = ... f ... in ... pattern

: 

In [25]:
/*--
let pfact = function (self) 
               function (n)
                  if (n == 0)
                  then 1
                  else 
                     n * ( self (self) (n-1) )
        --*/

val self = Ident("self")
val n = Ident("n")
val e1 = Mult(n, FunCall( FunCall(self, self), Minus(n, Const(1))))
val e2 = IfThenElse(Eq(n, Const(0)), Const(1), e1)
val e3 = FunDef("n", e2)
val e4 = FunDef("self", e3)
val e5 = Let("pfact", e4, FunCall(FunCall(Ident("pfact"), Ident("pfact")), Const(10)))

val prog = TopLevel(e5)

self: Ident = Ident("self")
n: Ident = Ident("n")
e1: Mult = Mult(
  Ident("n"),
  FunCall(FunCall(Ident("self"), Ident("self")), Minus(Ident("n"), Const(1.0)))
)
e2: IfThenElse = IfThenElse(
  Eq(Ident("n"), Const(0.0)),
  Const(1.0),
  Mult(
    Ident("n"),
    FunCall(
      FunCall(Ident("self"), Ident("self")),
      Minus(Ident("n"), Const(1.0))
    )
  )
)
e3: FunDef = FunDef(
  "n",
  IfThenElse(
    Eq(Ident("n"), Const(0.0)),
    Const(1.0),
    Mult(
      Ident("n"),
      FunCall(
        FunCall(Ident("self"), Ident("self")),
        Minus(Ident("n"), Const(1.0))
      )
    )
  )
)
e4: FunDef = FunDef(
  "self",
  FunDef(
    "n",
    IfThenElse(
      Eq(Ident("n"), Const(0.0)),
      Const(1.0),
      Mult(
        Ident("n"),
        FunCall(
          FunCall(Ident("self"), Ident("self")),
          Minus(Ident("n"), Const(1.0))
        )
      )
    )
  )
)
e5: Let = Let(
  "pfact",
  FunDef(
    "self",
    FunDef(
      "n",
      IfThenElse(
        Eq(Ident("n")

# Recursion in Lettuce

In this lecture, we will explore recursive functions in Lettuce. Thus far, we have defined functions in lettuce. As an example, we can define a program with a function call. 

```
    let f = function (x) x * x + 20
        in 
            f(20)
```

The program above defined a function that takes in an input x and returns x * x + 20. Thus, the call f(20) returns the vale 420. 

## Why do we need special handling for recursion? 

If we attempted to define a function such as factorial: 
```    
    let fact = function (x) 
                in (x <= 0)
                then 1
                else x * fact(x-1) 
         in fact(20)          
```            

This will lead to a problem due to the way we have been handling the let binding. 

Recall that to evaluate a let binding
    
       let x = <def_expr> in <body_expr> 
       
under the environment ENV, we proceed with the following steps: 
```

            * Evaluate <def_expr> under the environment ENV, let v be the value
            * evaluate the <body_expr> under the extended ENV + [x -> v]
                we can evalaute here because now we have all the elements 
```

The problem is that identifier x that is being defined is itself not in scope when we are evaluating <def_expr> 

Apply this logic to the recursive fact function (recalled again below): 

```
    let fact = function (x) 
                if (x <= 0)
                then 1
                else x * fact(x - 1) 
                
           in 
               fact(10) 

__just trying to make sense of this in your head of why it isn't working__
               
let y = 10 in 
    let foo = function (x) 
                x + y 
            in 
             foo(1) 
             
 1. Env = [y -> 10] 
 2. Env = [y -> 10, Closure(Ident("x"), Plus(Ident("x"),Ident("y"), env_o[y->10]
 
 so when we evaluate the fun call we have to just check if everything is ok and good. which it is then we must evaluate after extending the environment
     newEnv = env_o + [Ident("x")
 
 3. eval(Plus(Ident("x"), Ident("y")), new_env[x -> 1, y -> 10 ]) 
     
     therefore this works. 
     
Now we can do this same process and see why recursion doesn't work. 

So we have 

    let fact = function (x) 
        if (x <= 0)
        then 1
        else x * fact(x - 1) 
                in
                    fact(10) 
        
1. so we must first try and defined the function and make sure its a closure. 
    
    env = []
    FunDef("x", <func_body>, oldEnv[Closure(<func_body>, env_old = [] ) 
    
    now we have captured the closure and we will attempt to do a function call. 
    
    FunCall("fact", Const(10))
    
2.  we will find that "fact" =>  closure(<func_body> , env_old = []) 
    
    now we will try to evaluate, remember we have to extend the environment. 
    
    
      extended_env = [x -> 10] 
      
      We can see here that 'fact' (the one inside of the function body) isn't defined so we will get an problem. 

```



Notice that the identifier 'fact' occurs in the <def_expr> part of the definition as well, where it is not going to be part of the environment. Thus our interpreter thus far will reject this program during runtime. While evaluating the sub expression

            FunCall(Ident("fact", Minus(Ident("x") , Const(1)))

## Handling Recursion

We will now discuss how to handle recursion properly. To enable this, we will need to treat recursive definition different from regular definitions. 

Thus in our new version, we will use 'let rec' construct that tells us that whatever we are going to define is going to be recurisve. 

```
    let rec fact = function (x) 
                   if (x <= 0) 
                   then 1 
                   else x * fact(x - 1) 
                   
            in 
                    fact(10) 
```

The 'let rec' construct is very similar to a let binding but with on ebig difference: The defining expression for let rec has to a function. For instance. 

```
    let rec multiply_by_10 = function (x) 
                               if ( x == 0 ) 
                               then 0
                               else 10 + multiply_by_10(x - 1)                                                              
```

We will not allow recursive definitions that are not functions, For instance;
    
        let rec x = 1 + x 
        
This makes no sense at all, and is disallowed. 

Therefore, we will extend Lettuce with the construct

```
    let rec <function_name_identifier> = function (<param_identifier>)
                                            function_body_exp
                                      in 
                                    body_expr
```

Let us extend the grammar of a stripped down version:


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) } & \text{argument 1 - function name, argument 2 - parameter}\\
 &&& \text{argument 3 - function definition expression, argument 4 - body expr} \\[5pt]
\end{array}$$


For example, we would like to represent the concrete syntax
```
    let rec f = function (x) 
                if (0 >= x) 
                then 1
                else 1 + f(x - 1) 
                
            in 
                f(10)

As the following abstract syntax: 

    LetRec("f", "x", IfThenElse(
                        Geq(Const(0), Ident("x")), 
                        Const(1), 
                        Plus(Const(1), FunCall(Ident("f"), Minus(Ident("x"), Const(1)))))
                                    , FunCall(Ident("f"), Const(10))
                
```

As an excercise, match various parts of the abstract syntax with the concrete syntax as above. 



In [1]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
/* Here is the new stuff */

// functionName, Parameter, FunctionBodyExpression, "in" expression
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr

case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

In [2]:
val p1 = TopLevel(LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         ))

p1: TopLevel = TopLevel(
  LetRec(
    "f",
    "z",
    IfThenElse(
      Geq(Const(0.0), Ident("z")),
      Const(1.0),
      Plus(Const(1.0), FunCall(Ident("f"), Minus(Ident("z"), Const(1.0))))
    ),
    FunCall(Ident("f"), Const(10.0))
  )
)

## Handing Recursion: Using a Y-Combinator. 

We will explain an elegant approach to handling recursion using an idea called the Y-Combinator. To be sure, it is a theoretical idea but it's elegance is mind blowing, and therefore it is worth learning just for that. 

To begin with, let us just deal with the recursive version of factorial

```
    let factorial = function (n) 
                if (n == 0) 
                then 1 
                else n * factorial(n-1)
               
                    in ...
```
This will not work in our current version of Lettuce because, when evaluating the body of the factorial function, we do not have factorial in our environment. 

To make sure of this, we will cut and paste our original interpter below and if ever we see recursion, we will blow our top and throw an exception for now. 

In [3]:
/*-- CUT AND PASTE CODE FROM NOTEBOOK on Functions --*/
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
       
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
      
    
      
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}


defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined function evalExpr
defined function evalProgram

Let us naievly try to write down that function and see if it will run

In [4]:
/* -- 
let factorial = function (n) 
            if (n == 0) 
            then 1
            else n * factorial(n-1)
   in 
    factorial(10)
--*/

val fact = Ident("factorial")
val n = Ident("n")
val ite = IfThenElse(Eq(n, Const(0)), Const(1), Mult(n, FunCall(fact, Minus(n, Const(1))))) // if n == 0 then 1 else n * factorial(n-1)
val fundef = FunDef("n", ite)
val e = Let("factorial", fundef, FunCall(fact, Const(10)))
val wrongFactorial = TopLevel(e)


fact: Ident = Ident("factorial")
n: Ident = Ident("n")
ite: IfThenElse = IfThenElse(
  Eq(Ident("n"), Const(0.0)),
  Const(1.0),
  Mult(Ident("n"), FunCall(Ident("factorial"), Minus(Ident("n"), Const(1.0))))
)
fundef: FunDef = FunDef(
  "n",
  IfThenElse(
    Eq(Ident("n"), Const(0.0)),
    Const(1.0),
    Mult(Ident("n"), FunCall(Ident("factorial"), Minus(Ident("n"), Const(1.0))))
  )
)
e: Let = Let(
  "factorial",
  FunDef(
    "n",
    IfThenElse(
      Eq(Ident("n"), Const(0.0)),
      Const(1.0),
      Mult(
        Ident("n"),
        FunCall(Ident("factorial"), Minus(Ident("n"), Const(1.0)))
      )
    )
  ),
  FunCall(Ident("factorial"), Const(10.0))
)
wrongFactorial: TopLevel = TopLevel(
  Let(
    "factorial",
    FunDef(
      "n",
      IfThenElse(
        Eq(Ident("n"), Const(0.0)),
        Const(1.0),
        Mult(
          Ident("n"),
          FunCall(Ident("factorial"), Minus(Ident("n"), Const(1.0)))
        )
      )
    ),
    FunCall(Ident("factorial"), Const(10.0

In [5]:
evalProgram(wrongFactorial)

: 

Notice how the attempt to evaluate the factorial program fails exactly as predicted. How do we fix this? 

To do so, we can always just make a non recursive version like this: 

```
    let almost_factorial = function (f) 
                            function (n) 
                                if (n == 0) 
                                then 1 
                                elsen * f(n-1) 
                                
                                    in ....
        this should work because function (f) is in scope for function (n) ?? 
```
The idea here is to have a curried function 'almost_factorial' that takes in two arguments, an input function f and an argument n. The idea here is that if n == 0, then it returns 1, otherwise it calls f on n - 1

If f is any function that can safely compute a factorial, thsi would totally work, but we are left with a chicken and egg question: How do we get such a function in the first place? 

The key insight would be this: 
    * if we could defined 'almost_factorial' and jsut pass it as the argument f of the function, will it work? 
    
It would except we would have a type error. 
    * 'almost_factorial' needs two arguments but the function f being passed in only takes in one argument. 
    
    
Suppose we did the following minor change

```
    let almost_factorial = function (f) 
                            function (n) 
                                if ( n == 0 ) 
                                then 1
                                else n * f (f) (n-1)                        
                                
```

Convice yourself that the factorial function can be defined as: 

```
    let factorial = function (n) 
                    almost_factorial (almost_factorial) (n) 
                    
Eg,.

    factorial 0 = almost_factorial (0) = 1 
    
    factorial (1) = almost factorial (almost_factorial) (1) = 
                  = 1 * almost_factorial (almost_factorial) (0)
                  = 1
                  
    factorial (2) = almost_factorial (almost_factorial) (2) 
                  = 2 * almost_factorial (almost_factorial) (1)
                  = 2 
                  
                  
    This works!! Can we actually implement this and see if it works? 
        
        let almost_factorial = function (f)
                                function (n) 
                                  if (n == 0)
                                  then 1
                                  else n * f (f) (n-1) 
                                  
         in 
         let factorial = function (n)
                         almost_factorial (almost_factorial) (n) 
         in 
          factorial(10)
          
```

In [6]:
/*-- 
 Write the definition of almost_factorial
 directly into AST
 --*/

val factorial = Ident("factorial")
val almost = Ident("almost_factorial")
val n = Ident("n")
val f = Ident("f")

val e1 = FunCall(factorial, Const(10))

val double_af_call = FunDef("n", FunCall(FunCall(almost, almost), n))
val e2 = Let("factorial", double_af_call, e1)
val ite_expr = IfThenElse ( Eq(n, Const(0)), Const(1), Mult(n, FunCall(FunCall(f, f), Minus(n, Const(1)))) ) 
val  af_body = FunDef("f", FunDef("n", ite_expr))

val p10 = TopLevel( Let("almost_factorial", af_body, e2))


factorial: Ident = Ident("factorial")
almost: Ident = Ident("almost_factorial")
n: Ident = Ident("n")
f: Ident = Ident("f")
e1: FunCall = FunCall(Ident("factorial"), Const(10.0))
double_af_call: FunDef = FunDef(
  "n",
  FunCall(
    FunCall(Ident("almost_factorial"), Ident("almost_factorial")),
    Ident("n")
  )
)
e2: Let = Let(
  "factorial",
  FunDef(
    "n",
    FunCall(
      FunCall(Ident("almost_factorial"), Ident("almost_factorial")),
      Ident("n")
    )
  ),
  FunCall(Ident("factorial"), Const(10.0))
)
ite_expr: IfThenElse = IfThenElse(
  Eq(Ident("n"), Const(0.0)),
  Const(1.0),
  Mult(
    Ident("n"),
    FunCall(FunCall(Ident("f"), Ident("f")), Minus(Ident("n"), Const(1.0)))
  )
)
af_body: FunDef = FunDef(
  "f",
  FunDef(
    "n",
    IfThenElse(
      Eq(Ident("n"), Const(0.0)),
      Const(1.0),
      Mult(
        Ident("n"),
        FunCall(FunCall(Ident("f"), Ident("f")), Minus(Ident("n"), Const(1.0)))
      )
    )
  )
)
p10: TopLevel = TopLevel(
  Let(
    "alm

In [7]:
evalProgram(p10)

res6: Value = NumValue(3628800.0)

# Recipe for converting a recursive definition into a non-recursive one. 

Consider at the high level function rfun being defined recursively as follows: 


```
    let rec rfun = function (x) <f_expr> 
    in 
    b_expr
```

This is recursive, therefore, in the body f_expr, we have a recursivecalls back to rfun
```
    * Step 1: Replace all recursive calls with non recursive ones to a parameter function f. 
        let almost_rfun = function(f) function(x) <sub rfunin f_expr with f (f) > 
        in ...
        
    
    * Step 2: define the orginal function rfun as 
    
        let rfun = function (x) 
                    almost_rfun (almsot_rfun) (x) 
        in 
        ...
        
    This works and can be easily implmeented as a function call eliminateRecursion    
```

As an example, consider

```
    let rec crazy_rec = function (x)
                        if (x >= 0)
                        then 0.5 * crazy_rec (1 - x) 
                        else 20 * crazy_rec (0 - x) 
          in 
          crazy_rec( -25.0) 
    
    will be transofrmed into 
                                # the function (f) is essentially passing the function into the next stack
     let almost_crazy_rec = function (f)
                                     # the function (x) is essentially the argument of the function
                             function (x) 
                                 if (x >= 0) 
                                 then 0.5 * f (f) (1-x) 
                                 else 20 * f (f) (0 - x) 
                      in 
           let crazy_rec = function (x) 
               almsot_crazy_rec (almost_crazy_rec)(x) 
              
                  in 
                      crazy_rec (-25.0)
                      
```

In [8]:
/* Function substituteOccurrences replaces every where Identifier(oldX)
       occurs in e with the expression newX.
       We will need this function to convert recursion to non-recursive version.
       */
    
def substituteOccurrences(e: Expr, oldX: String, newX: Expr): Expr = { 
    // Define helper functions
     def applyTo (e1: Expr, e2: Expr) (res: (Expr,Expr) => Expr) = {
            res(substituteOccurrences(e1, oldX, newX), 
                substituteOccurrences(e2, oldX, newX)) 
     }
    
    def subs(e: Expr): Expr = substituteOccurrences(e, oldX, newX)
    // Now do the case matching
    e match {
   
        case Const(_) => e // Nothing to be done
        case Ident(x) => if (x == oldX) newX else e // This is where the substitution happens!!
        case Plus(e1, e2) => applyTo (e1, e2) (Plus(_, _)) 
        case Minus(e1, e2) => applyTo (e1, e2) (Minus(_, _)) 
        case Mult(e1, e2) => applyTo (e1, e2) (Mult(_, _)) 
        case Eq(e1, e2) => applyTo (e1, e2) (Eq(_, _))
        case Geq(e1, e2) => applyTo (e1, e2) (Geq(_, _))
        case IfThenElse(e, e1, e2) => IfThenElse( subs(e), subs(e1), subs(e2) )
        case Let(x, e1, e2) => {
            val e1Hat = subs(e1)
            if (x == oldX)
                Let(x, e1Hat, e2) // Do not substitute anymore since oldX is not re-bound to e1Hat and thus out of scope
            else
                Let(x, e1Hat, subs(e2))
        }
        
        case FunDef(x, e) => {
            if (x == oldX)
                FunDef(x, e) // Do not substitute since scope of oldX ends beacuse the function parameter shadows it
            else 
                FunDef(x, subs(e))
        }
        
        case FunCall(e1, e2) => {
            FunCall(subs(e1), subs(e2))
        }
        
        case LetRec(f, x, e1, e2) => {
            val e1Hat = if (x == oldX) e1 else subs(e1)
            val e2Hat = if (f == oldX) e2 else subs(e2)
            LetRec(f, x, e1Hat, e2Hat)
        }
    }
}

// Function: eliminateRecursion
//  Eliminate all let rec from expression e
// id is just used to generate fresh temporary names that we will need.

def eliminateRecursion(e: Expr, id: Int = 0): Expr = e match {    
    case Const(_) => e // Nothing to be done
    case Ident(_) => e // Nothing to be done
    case Plus(e1, e2) => Plus( eliminateRecursion(e1), eliminateRecursion(e2))
    case Minus(e1, e2) => Minus ( eliminateRecursion(e1), eliminateRecursion(e2))
    case Mult(e1, e2) => Mult( eliminateRecursion(e1), eliminateRecursion(e2))
    case Eq(e1, e2) => Eq( eliminateRecursion(e1), eliminateRecursion(e2))
    case Geq(e1, e2) => Geq(  eliminateRecursion(e1), eliminateRecursion(e2))
    case IfThenElse(e, e1, e2) => IfThenElse( eliminateRecursion(e), eliminateRecursion(e1), eliminateRecursion(e2))
    case Let(x, e1, e2) => Let(x, eliminateRecursion(e1), eliminateRecursion(e2) )
    case FunDef(x, e) => FunDef(x, eliminateRecursion(e))
    case FunCall(e1, e2) => FunCall(eliminateRecursion(e1), eliminateRecursion(e2))
    case LetRec(rfun, x, fExpr0, bExpr0) => {
        /*-- This is where action is 
            (a) substitute a new fresh string "almost_rfun" wherever rfun is referenced in fExpr
            (b) define the function call almost_rfun
            (c) define rfun as a simple function call in terms of almost_rfun
        --*/
        val fExpr = eliminateRecursion(fExpr0, id + 1)
        val bExpr = eliminateRecursion(bExpr0, id + 2)
        val almost_name = "almost_"+rfun // Name of new function -- can be more sophisticated in choosing a name
        val almost = Ident(almost_name)
        val fun_defn = Let(rfun, FunDef(x, FunCall(FunCall(almost, almost), Ident(x))), bExpr)
        val tmp_fun_name = "rfun_param_tmp__" + id.toString + "__"
        val tmp_expr = FunCall(Ident(tmp_fun_name), Ident(tmp_fun_name))
        val new_f_expr = substituteOccurrences(fExpr, rfun, tmp_expr) // Carry out the substitution
        val almost_rfun_defn = FunDef(tmp_fun_name, FunDef(x, new_f_expr))
        val almost_rfun = Let(almost_name, almost_rfun_defn, fun_defn)
        almost_rfun
    }
}




defined function substituteOccurrences
defined function eliminateRecursion

In [9]:
val e1 = LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         )
val p1 = TopLevel(e1)

val p2 = TopLevel( 
            eliminateRecursion(e1)
         )
    

e1: LetRec = LetRec(
  "f",
  "z",
  IfThenElse(
    Geq(Const(0.0), Ident("z")),
    Const(1.0),
    Plus(Const(1.0), FunCall(Ident("f"), Minus(Ident("z"), Const(1.0))))
  ),
  FunCall(Ident("f"), Const(10.0))
)
p1: TopLevel = TopLevel(
  LetRec(
    "f",
    "z",
    IfThenElse(
      Geq(Const(0.0), Ident("z")),
      Const(1.0),
      Plus(Const(1.0), FunCall(Ident("f"), Minus(Ident("z"), Const(1.0))))
    ),
    FunCall(Ident("f"), Const(10.0))
  )
)
p2: TopLevel = TopLevel(
  Let(
    "almost_f",
    FunDef(
      "rfun_param_tmp__0__",
      FunDef(
        "z",
        IfThenElse(
          Geq(Const(0.0), Ident("z")),
          Const(1.0),
          Plus(
            Const(1.0),
            FunCall(
              FunCall(
                Ident("rfun_param_tmp__0__"),
                Ident("rfun_param_tmp__0__")
              ),
              Minus(Ident("z"), Const(1.0))
            )
          )
        )
      )
    ),
    Let(
      "f",
      FunDef(
        "z",
        F

In [10]:
evalProgram(p2)

res9: Value = NumValue(11.0)

In [10]:
val e7=  LetRec("fact", "n", 
          IfThenElse(Eq(Ident("n"), Const(0)), Const(1), 
                     Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1))))
                    ),
           FunCall(Ident("fact"), Const(7))
          )

val fact_rec_7 = TopLevel(
   e12
)

val fact_no_rec_7= TopLevel( eliminateRecursion(e7) )

cmd10.sc:9: not found: value e12
   e12
   ^Compilation Failed

: 

In [10]:
evalProgram(fact_no_rec_7)

cmd10.sc:1: not found: value fact_no_rec_7
val res10 = evalProgram(fact_no_rec_7)
                        ^Compilation Failed

: 

# Handling Recursion using Environments

As we already saw, one strategy to handle recursion is by rewriting the recursive call to elimnate recursion. Another approach is to directly write an interpretter than handles recursion. However, the problem is that if we are looking at a recurisve call: 

    let rec f = funciton (x) <body of function expr> in <def.Expr> 
    
The main problem to solve is that inside of <body of function expr> everytime we see a reference to f, we should be able to resolve it to the recusive call itself. 

Let us consider how we handl ea normal non-recursive definiton 

    let f = function (x) <body_expr> in <def_expr>
    
under the environment ENV, we evaluate the function body to the closure that encapsulates three things: Closure(<params> , <function body> , oldEnv) 

Remember that this does not work for recursive functions f since when we execute a recursive call to f in the <body of function expr> , it is not defined in the environment ENV. 



Therefore, the strategy is to extend $\sigma$ into a new environent $\color{red}{\hat{\sigma}}$ as follows:
- $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$.
- $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$

I.e, $\color{red}{\hat{\sigma}}$ defines the function f as a closure and notice that the environment of thenew closure again! This means that if we call f in the body in a function expr, we get the same environment with just the formal parameter of x updated. This gives us what we need to implement recursion: An environmet that keeps defining f to the appropriate function


Thus we are going to reconsider environments from scratch? 




## Environments

What is an environment? We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered? 

    * The empty environment: implemented as an empty map. Let us call this: EmptyEnv 
    
    * The environment ENV[x -> v] which denotes the previously existing environment extedned with the mapping x->v that associates identfier x with value v. Let us call this operation : Extend(ENV, x, v) 
    
Let us add a third kind of extension to support recursion: 

    * ExtendRec(f,x,e,ENV) which creates a newENV such that all previously defined mappings exist, but we extend it with: Closure(x,<body of function expr>, oldENV) 
    
        Remember that this works because here the function call has access to all elements but we are allowing some identifer (x) be updated when we evaluate the closure in the next function call. 
        
        

In [11]:
sealed trait Environment 
sealed trait Value

case object EmptyEnv extends Environment
case class Extend(x: String, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment

/* -- We need to redefine values to accomodate the new representation of environments --*/
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: Expr, pi: Environment) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


/*-- Operations on environments --*/

def lookupEnv(sigma: Environment, x: String): Value = sigma match {
    case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, _) if y == x => v
    case Extend(_, _, pi) => lookupEnv(pi, x)
    case ExtendRec(f, y, e, pi) => if (x == f) 
                                          Closure(y, e, sigma)
                                   else
                                          lookupEnv(pi, x)
}


defined trait Environment
defined trait Value
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined function lookupEnv

In [12]:
/*-- We can rewrite evalExpr now to handle recursion as well --*/
def evalExpr(e: Expr, env: Environment): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f) // Same as before
        
        case Ident(x) => lookupEnv(env, x) // Changed to accomodate the new environment definitions.
    
        /* Ditto as before */
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
        /* Ditto as before */
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
        /* Ditto as before */
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        /* Ditto as before */
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        /* Ditto as before */
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        /* Ditto as before */
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        /* Ditto as before */
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = Extend(x, v1, env) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
        /* Ditto as before */
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        /* Ditto as before */
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = Extend(x, v2, closed_env)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        /*-- This is where we handle recursion --*/
        case LetRec(rfun, x, fExpr, bExpr) => {
            val env2 = ExtendRec(rfun, x, fExpr, env)
            evalExpr(bExpr, env2)
        }
    }
}

def evalProgram(p: Program) = {
    p match { 
        case TopLevel(e) => evalExpr(e, EmptyEnv)
    }
}


defined function evalExpr
defined function evalProgram

In [13]:
val fact_rec_8 = TopLevel(
    LetRec("fact", "n", 
          IfThenElse(Eq(Ident("n"), Const(0)), Const(1), 
                     Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1))))
                    ),
           FunCall(Ident("fact"), Const(8))
          )
)
evalProgram(fact_rec_8)

fact_rec_8: TopLevel = TopLevel(
  LetRec(
    "fact",
    "n",
    IfThenElse(
      Eq(Ident("n"), Const(0.0)),
      Const(1.0),
      Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1.0))))
    ),
    FunCall(Ident("fact"), Const(8.0))
  )
)
res12_1: Value = NumValue(40320.0)

In [14]:
/*--
let rec crazy_rec = function (x)
                    if (x >= 0)
                    then 0.5 * (1 - x)
                    else 20 * crazy_rec ( 0 - x )
 in 
 crazy_rec( - 25.0)
 --*/

val crec = Ident("crazy_rec")
val x = Ident("x")

val e1 = FunCall(crec, Const(-25.0))
val e21 = Mult(Const(0.5), Minus(Const(1), x))
val e22 = Mult(Const(20.0), FunCall(crec, Minus(Const(0), x)))
val e2 = LetRec("crazy_rec", "x", IfThenElse(Geq(x, Const(0)), e21, e22), e1)

val crazyProg = TopLevel(e2)

crec: Ident = Ident("crazy_rec")
x: Ident = Ident("x")
e1: FunCall = FunCall(Ident("crazy_rec"), Const(-25.0))
e21: Mult = Mult(Const(0.5), Minus(Const(1.0), Ident("x")))
e22: Mult = Mult(
  Const(20.0),
  FunCall(Ident("crazy_rec"), Minus(Const(0.0), Ident("x")))
)
e2: LetRec = LetRec(
  "crazy_rec",
  "x",
  IfThenElse(
    Geq(Ident("x"), Const(0.0)),
    Mult(Const(0.5), Minus(Const(1.0), Ident("x"))),
    Mult(
      Const(20.0),
      FunCall(Ident("crazy_rec"), Minus(Const(0.0), Ident("x")))
    )
  ),
  FunCall(Ident("crazy_rec"), Const(-25.0))
)
crazyProg: TopLevel = TopLevel(
  LetRec(
    "crazy_rec",
    "x",
    IfThenElse(
      Geq(Ident("x"), Const(0.0)),
      Mult(Const(0.5), Minus(Const(1.0), Ident("x"))),
      Mult(
        Const(20.0),
        FunCall(Ident("crazy_rec"), Minus(Const(0.0), Ident("x")))
      )
    ),
    FunCall(Ident("crazy_rec"), Const(-25.0))
  )
)

In [15]:
evalProgram(crazyProg)

res14: Value = NumValue(-240.0)

## Lecture notes

He is going over how to rewrite a factorial function, this is a y-combinator


Every function has a function called "what should i do next' 
every function takes two arguments. 

let f = function (WhatNext)  <-- this is how we are going to be be able call the function again. 
                   function (x) <-- this is our actual argument 
                       if (x <= 1) 
                       then 1
                       else x * WhatNext(WhatNext) (x-1) 
                       
                 in 
                   let factorial = f(f) 
                       in factorial (10)       # this is essentially,  factorial = f(f)(10)

you essentially defined f, but didn't use it anywwhere in the body function. 


letrec(string, string, expr) 
       <name of function> <name of parameters> <body of the function given> 



## let rec binding

LetRec(rfun, x, fExpr, bExpr)

With this we are able to have our function be in scope the defining expression. When we use this, it doesn't stop a programmer from writing a program like this. 

```
    let rec x = x - 1
        in 
            x - 2               We will rule away programs like this. 
            
What about this one

    let rec fact = function(x) 
                     fact(x).         this is an infinite loop, we will also rule these away. 
```

The halting problem: You cannot write a program to tell if another program contains an infinute loop or not. 


Our current Abstract Syntax

Expr => Const(Double) 
        Ident(String) 
        Plus(Expr, Expr) 
        If .. then.. else
        Let(String, Expr, Expr) 
        FunDef(String, Expr) 
        FunCall(Expr, Expr) 
        
               <funcName> <formalParameter> <funcBody> <bodyFunc> 
        letrec(String, String, Expr, Expr) 
        
        
        
Example.
```
                          if then else    FunCall(Ident...)
letRec("factorial", "x", <functionBody>, factorial(10))
```



# Circuluar (rec) Scope Chains

__Environemnt__: Map[String, Value]
    
```
    let x = 10                    [x -> 10]
        in 
            let y = 15 in           [x -> 10, y ->15]
                let x = y in           [y -> 15, x -> 15]
                    x + y                  30
```

Now lets view environments as linked lists. When we view them like this we call them chains. Really good because they can grow / shrink abritraily. 


We will write the same environment the same as above, but with a linked list. 

Start with an empty list.  Recent encountered elements go in the front

 [x:15] -> [y:15] -> [x:10] -> Nil
 
 The rule is processing from LEFT to RIGHT, look up the first value encountered for what you are looking for then return it back
 

What happens if look for something not in the scope chain? We traverse the list then throw an error if we hit Nil. 

Why bother implementing with linked list? Lets look at how it would handle this in our factorial function. 


```
    let rec factorial = function(x) 
                        if (n <= 1)
                        then 1
                        else x * factorial(x - 1) 
          in
```

Closure("x", if(n <= 1) then 1 else x * factorial(x - 1), Make it a circular pointing to the front of the list) 

In [25]:
sealed trait Expr
sealed trait Value
case class NumValue(d: Double) extends Value
case class Closure(s:String, e: Expr, env: Environment)


sealed trait Environment
case object emptyEnvironment extends Environment
case class Extend(x: String, v: Value, rest: Environment) extends Environment
case class ExtendRec(x: String, param: String, body: Expr, rest: Environment) extends Environment

defined trait Expr
defined trait Value
defined class NumValue
defined class Closure
defined trait Environment
defined object emptyEnvironment
defined class Extend
defined class ExtendRec

In [23]:
// (x : 15) -> (y : 15) -> (x: 10) -> Nil


Extend("x", NumValue(15.0), Extend("y", NumValue(15.0), Extend("x", NumValue(15.0), emptyEnvironment)))

res22: Extend = Extend(
  "x",
  NumValue(15.0),
  Extend("y", NumValue(15.0), Extend("x", NumValue(15.0), emptyEnvironment))
)

In [25]:
def lookup(x: String, env: Environment): Value = env match {
    
    case emptyEnvironment => throw new IllegalArgumentException(s"How dare you ask me too look up a name that dones not exist")
    case Extend(s, v, rest) => {
        if (x == s){
            // found it 
            v
        } else{
            lookup(x, rest)
        }
    }
    case ExtendRec(fun, param, exprBody, rest) => {
        if (x != fun){
            lookup(x, rest) 
        }else{
            Closure(param, exprBody, env)
        }
    }
}

cmd25.sc:16: type mismatch;
 found   : cmd25.this.cmd24.Closure
 required: cmd25.this.cmd24.Value
            Closure(param, exprBody, env)
                   ^Compilation Failed

: 

## Finishing the topic of recursion
__Before we had: __ 
    Environment:  String -> Value, a Map in scala
    
    now we define environments as grammar. 
    
    Environment => EmptyEnvironment
                   Extend(String, Value, Environment) 
                   ExtendRec(String, String, Expr, Environment) 
                   
Then we created a function
    
    lookup(x: String, env: Environment) : Value
    
__(Example of a scope chain)__

    [x : 10] -> [y :true] -> [f : Closure(...)] -> [x : 20] -> Nil

__In abstract Syntax we have__

    Extend("x", NumValue(10), 
            Extend("y", BoolValue("true"), 
                Extend("f", Closure..., 
                    Extend("x", NumValue(20), 
                            EmptyEnvironment))))
                            
__looking back at the lookup function__
    It said give me a string and an environment and i will return you a Value. To do this we essentially just traversed the entire thing. 
    
    If we did lookup("y", env),  it will return true
    
__the function returns the first one found__

If we ignore the extendRec, the function is a case matching pattern 

    lookup(x: String, env: Environment): Value = env match{
    
        case emptyEnvironment => throws an exception here
        
        caes Extend(s, v, rest) => {
            
               // if we find the value inside of our environment
            if (x == s){
                return v
            }
            
                // we didn't find it so we search the rest 
            else{
                lookup(x, rest)
            }
        }
        
        case ExtendRec(nameOfFunc, formalParam, bodyExpr, restOfEnv) => {
            
            if (x == nameOfFun){
            
                   // this is what makes the recursion work. 
                Closure(formalParam, bodyExpr, env)
            }
            
            else{
                lookup(x, restOfEnv) 
            }
        
        }
    }
    
__lets try to make the extendRec part make more sense__
remember that the whole point of this is just creating a new environment. 


let y = 15 in 
    let rec foo = function (x) 
                    foo(x - 1) + y
             in 
             let y = 20 in 
                 foo(y)                  we are going to trace this program and go through the scope chain
                 
```
1.
    emptyEnv
    Extend(y, 15, emptyEnv) 
        [y: 15] -> Nil

2. 
    ExtendRec("foo", "x", body, env) 
    
            [ExtendRec] -> [y:15] -> Nil
            
3.  Extend("y", 20, env) 

           finalEnv =  [y:20] -> [ExtendRec] -> [y:15] -> Nil
            
            
Now we try to start evaluating the function. 
    starting by doing a funCall foo(y) 
    
            finalEnv =  [y:20] -> [foo: ExtendRec] -> [y:15] -> Nil
    
Q1: What happens when we lookup foo in our environment (hint: It should evaluate to a closure) 
        Closure("x", body, env2) 
Q2: What happens when we lookup y
        20
```